In [13]:
import random
import pronouncing

def build_markov_chain(text, order=1):
    """Build a Markov chain with rhyming capabilities 🎶"""
    words = text.split()
    markov_chain = {}

    for i in range(len(words) - order):
        key = tuple(words[i:i+order])
        next_word = words[i+order]
        if key not in markov_chain:
            markov_chain[key] = []
        markov_chain[key].append(next_word)

    return markov_chain

def generate_rhyming_markov_text(chain, length=10, seed_word=None):
    """Generate rhyming text using Markov chains + rhyming words 🎤"""
    key = random.choice(list(chain.keys())) if not seed_word else seed_word
    generated_words = list(key)

    for _ in range(length):
        if key in chain:
            possible_words = chain[key]
            rhyming_choices = [w for w in possible_words if pronouncing.rhymes(w)]
            next_word = random.choice(rhyming_choices) if rhyming_choices else random.choice(possible_words)
            generated_words.append(next_word)
            key = tuple(generated_words[-len(key):])
        else:
            break

    return ' '.join(generated_words)

# 📜 Example text with poetic vibes:
song_lyric = """
Search for the answers I knew all along.
I lost myself we all fall down.
Alone, I stand.
A broken man.
"""

# 🏗️ Build & Generate!
markov_chain = build_markov_chain(song_lyric, order=2)
rhyming_text = generate_rhyming_markov_text(markov_chain, length=10)

print("🎶 Rhyming Markov Generated Lyric:")
print(rhyming_text + " ✨")

🎶 Rhyming Markov Generated Lyric:
stand. A broken man. ✨


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import numpy as np

# 📜 Custom text dataset
song_lines = [
"All I have is one last chance",
"I won't turn my back on you",
"Take my hand",
"Drag me down",
"If you fall, then I will, too",
"And I can't save what's left of you",
"Sing something new",
"I have nothing left",
"I can't face the dark without you",
"There's nothing left to lose",
"The fight never ends",
"I can't face the dark without you"
]

# 🌟 Tokenize the dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(song_lines)
total_words = len(tokenizer.word_index) + 1

# 📊 Convert text into training sequences
input_sequences = []
for line in song_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

# 🚀 Pad sequences for consistency
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# 🎯 Define features & labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# 🔥 Build an LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 8, input_length=max_sequence_length-1),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

# ⚡ Compile & Train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=1)

# 🏆 Generate new poetic lines!
def generate_song(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        seed_text += " " + tokenizer.index_word[predicted[0]]
    return seed_text

# 📝 Example Run
print("\n📜 AI-Generated Poetry:")
print(generate_song("pull me apart", next_words=5))

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.0230 - loss: 3.7844
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0126 - loss: 3.7823    
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1276 - loss: 3.7807
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1276 - loss: 3.7785
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1506 - loss: 3.7764
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1150 - loss: 3.7740
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0941 - loss: 3.7721
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1046 - loss: 3.7694
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1067 - loss: 3.7667
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1046 - loss: 3.7639
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1046 - loss: 3.7586
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0941 

In [ ]:
from transformers import pipeline
import random
# import tf_keras as keras


# 🚀 Load AI wizard (GPT-2)
rnn_generator = pipeline("text-generation", model="gpt2")

# 🧙‍♂️ Markov Chain Generator
def generate_markov_seed(chain, length=5):
    """Generate a Markov chain text snippet to feed into GPT-2"""
    key = random.choice(list(chain.keys()))
    generated_words = list(key)

    for _ in range(length):
        if key in chain:
            next_word = random.choice(chain[key])
            generated_words.append(next_word)
            key = tuple(generated_words[-len(key):])
        else:
            break

    return ' '.join(generated_words)

# 🏗️ Build Markov Chain
# Song Without You by Breaking Benjamin
song_lyric = """
Sing something new
I have nothing left
I can't face the dark without you
There's nothing left to lose
The fight never ends
I can't face the dark without you
Swallow me under and pull me apart
I understand, there's nothing left
Pain so familiar and close to the heart
No more, no less, I won't forget
Come back down, save yourself
I can't find my way to you
And I can't bear to face the truth

"""
markov_chain = build_markov_chain(song_lyric, order=4)

# 🔮 Generate a Markov seed & complete with RNN magic!
markov_seed = generate_markov_seed(markov_chain, length=7)
rnn_output = rnn_generator(markov_seed, max_length=50, num_return_sequences=1)

print("💡 Hybrid AI Poet Generated Text:")
print(rnn_output[0]["generated_text"] + " ✨")